In [1]:
import pandas as pd
pd.options.display.max_columns = None

data = pd.read_csv('data/train.csv')

In [2]:
import regex as re

In [3]:
data.shape

(4167, 74)

Comenzaremos eliminando las columnas que decidimos descartar de mara directa en la exploración inicial.

In [4]:
columns = ['neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated', 'listing_url', 'scrape_id', 'last_scraped',
           'picture_url', 'host_id', 'host_url', 'host_thumbnail_url', 'host_picture_url', 'calendar_last_scraped', 'license']

data.drop(columns, axis=1, inplace=True)

In [5]:
data.shape

(4167, 61)

Cuanto mayor sea el número de columnas, más complicado será trabajar con ellas. Por eso lo que haremos será reducirlas mucho (aun sabiendo que algunas de las que descartamos pueden aportar valor a nuestro modelo) y a partir de ahí trataremos de afinar el modelo incluyendo algunas de esas columnas eliminadas (si lo considerásemos oportuno).

Empezamos dropeando las columnas que sospechamos que no aportan valor (sin entrar en profundidad en los datos, ya sea por la información que aportan los datos o por el formato en el que se nos proporcionan). También haremos algunas comprobacionas con las que tengamos dudas más específicas.

In [6]:
columns = ['name', 'description', 'neighborhood_overview', 'host_name', 'host_location', 'host_about', 'host_listings_count',
          'neighbourhood', 'property_type', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
          'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm',
          'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365',
          'number_of_reviews_ltm', 'number_of_reviews_l30d', 'review_scores_accuracy', 'review_scores_cleanliness',
          'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value',
          'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes',
          'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms']

cleaning = data.drop(columns, axis=1)

In [7]:
cleaning.shape

(4167, 28)

In [8]:
cleaning.columns

Index(['id', 'host_since', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'room_type',
       'accommodates', 'bathrooms_text', 'bedrooms', 'beds', 'amenities',
       'price', 'minimum_nights', 'number_of_reviews', 'first_review',
       'last_review', 'review_scores_rating', 'instant_bookable',
       'reviews_per_month'],
      dtype='object')

Una vez hecha la limpieza general, vamos fila por fila para transformar los datos según consideremos oportuno.

**'host_since'**: los valores son fechas en tipo string. Lo que haremos será establecer como valor el año (en formato int) en lugar de la fecha completa. Esto lo hacemos en base a la creencia de que el año de registro del host influye en el precio esablecido.

In [9]:
cleaning.host_since[0]

'2012-07-24'

In [10]:
cleaning.host_since = cleaning.host_since.apply(lambda x: int(x[:4]))

In [11]:
cleaning.host_since.value_counts()

2014    705
2013    641
2015    585
2016    511
2012    407
2019    303
2018    289
2017    252
2020    170
2021    146
2011    125
2010     29
2009      3
2008      1
Name: host_since, dtype: int64

**'host_response_time'**: tiene cuatro valores únicos que indican la rapidez de respuesta del host. Podemos entenderlo como una escala ordinal por lo que lo trataremos como tal. Es cierto que este orden no debería ser homogéneo, es decir, no es lo mismo el salto de una hora a unas pocas horas que de un día a varios días o más. No tenemos una base real para esto, por lo que simplificaremos bastante esta distinción. Lo que haremos será "penalizar" a aquellos hosts que responden en varios días o más (entendemos que es el tiempo de respuesta al que el usuario es más sensible).

In [12]:
cleaning.host_response_time.value_counts()

within an hour        1550
within a few hours     573
within a day           556
a few days or more      75
Name: host_response_time, dtype: int64

In [13]:
cleaning.host_response_time = cleaning.host_response_time.replace(['within an hour', 'within a few hours', 'within a day', 'a few days or more'],
                                    [1, 2, 3, 5])

Una vez hecha la transformación nos damos cuenta de que la cantidad de NaN es excesiva y no tendría mucho sentido definir valores sin saber cuál es realmente el tiempo de respuesta (al menos, según nuestro planteamiento). Entedemos que la presencia de NaN no es algo que dependa del host o de la propia plataforma Airbnb, sino que se debe a fallos en la extracción de los datos, lo cual complica aún más el manejo de los valores nulos. Decidimos eliminar la columna 'host_response_time'.

In [14]:
cleaning.host_response_time.isnull().sum()

1413

In [15]:
cleaning.drop('host_response_time', axis=1, inplace=True)

**'host_response_rate'**: siguiendo el planteamiento anterior (gran proporción de NaN y dificultad para gestionarlos), decidimos eliminar esta columna.

In [16]:
cleaning.host_response_rate.isnull().sum()

1413

In [17]:
cleaning.drop('host_response_rate', axis=1, inplace=True)

**'host_acceptance_rate'**: en la linea de lo anterior y teniendo en cuenta que ya dudábamos de la aportación de valor de esta columna, también decidimos eliminar esta columna.

In [18]:
cleaning.host_acceptance_rate.isnull().sum()

1092

In [19]:
cleaning.drop('host_acceptance_rate', axis=1, inplace=True)

**'host_is_superhost'**: los valores son 'f' (false) o 't' (true) en formato string. No hay NaN ni ningún otro valor. Simplemente lo que haremos será sustituir 'f' por 0 y 't' por 1 (en formato int lógicamente).

In [20]:
cleaning.host_is_superhost.value_counts()

f    3081
t    1086
Name: host_is_superhost, dtype: int64

In [21]:
cleaning.host_is_superhost = cleaning.host_is_superhost.replace(['f', 't'], [0, 1])

In [22]:
cleaning.host_is_superhost.value_counts()

0    3081
1    1086
Name: host_is_superhost, dtype: int64

**'host_neighbourhood'**: nos damos cueta de que ya existe otra columna que hace referencia al barrio ('neighbourhood_cleansed') y además esta otra columna tiene un formato más amigable (22 valores únicos). Decidimos eliminar esta columna.

In [23]:
len(cleaning.host_neighbourhood.unique())

52

In [24]:
len(cleaning.neighbourhood_cleansed.unique())

22

In [25]:
cleaning.drop('host_neighbourhood', axis=1, inplace=True)

**'host_total_listings_count'**: nos indica la cantidad de propiedades listadas por el host. Entendemos que esto puede influir en el precio ya que, a mayor número de propiedades, más competitivo podrá ser el precio (especialmente si se trata de inmobiliarias o similares). No contiene valores nulos, por lo que mantenemos la columna como está.

In [26]:
cleaning.host_total_listings_count.isnull().sum()

0

**'host_verifications'**: es una columna bastante incómoda de tratar. Viene en formato lista y contiene (o no) diferentes valores en cada caso. Lo que haremos será hacer un recuento de los diferentes tipos de verificación para ese host y establecer este recuento como valor (entendiendo que a mayor cantidad, mayor fiabilidad o facilidad para el usuario).

In [27]:
def limpieza_lista(x):
    x = x.lstrip("['")
    x = x.rstrip("']")
    x = x.split("', '")
    return len(x)

In [28]:
cleaning.host_verifications = cleaning.host_verifications.apply(limpieza_lista)

In [29]:
cleaning.host_verifications.value_counts()

4     700
5     629
8     553
7     537
2     500
6     489
3     423
9     191
1      90
10     45
11     10
Name: host_verifications, dtype: int64

**'host_has_profile_pic'**: nos damos cuenta de que sólo hay 12 valores 'f' y el resto son 't'. Esto no aportará valor a nuestro modelo, por lo que eliminamos la columna.

In [30]:
cleaning.host_has_profile_pic.value_counts()

t    4155
f      12
Name: host_has_profile_pic, dtype: int64

In [31]:
cleaning.host_has_profile_pic.isnull().sum()

0

In [32]:
cleaning.drop('host_has_profile_pic', axis=1, inplace=True)

**'host_identity_verified'**: seguimos el mismo planteamiento que en 'host_is_superhost', sustituyendo los valores que son 'f' 'f' por 0 y los 't' por 1 (en formato int lógicamente).

In [33]:
cleaning.host_identity_verified.value_counts()

t    3355
f     812
Name: host_identity_verified, dtype: int64

In [34]:
cleaning.host_identity_verified = cleaning.host_identity_verified.replace(['f', 't'], [0, 1])

In [35]:
cleaning.host_identity_verified.value_counts()

1    3355
0     812
Name: host_identity_verified, dtype: int64

**'cleaning.neighbourhood_cleansed'**: contiene 22 valores únicos referentes al barrio. Generaremos variables ficticias a través de un get_dummies.

In [36]:
len(cleaning.neighbourhood_cleansed.unique())

22

In [37]:
cleaning.neighbourhood_cleansed.isnull().sum()

0

In [38]:
cleaning.neighbourhood_cleansed.value_counts()

Centrum-West                              639
De Baarsjes - Oud-West                    611
Centrum-Oost                              470
De Pijp - Rivierenbuurt                   423
Zuid                                      275
Westerpark                                266
Oud-Oost                                  241
Bos en Lommer                             193
Oud-Noord                                 188
Oostelijk Havengebied - Indische Buurt    166
IJburg - Zeeburgereiland                  111
Watergraafsmeer                           108
Noord-West                                 88
Noord-Oost                                 76
Slotervaart                                68
Geuzenveld - Slotermeer                    58
Buitenveldert - Zuidas                     44
De Aker - Nieuw Sloten                     38
Gaasperdam - Driemond                      32
Osdorp                                     29
Bijlmer-Centrum                            28
Bijlmer-Oost                      

In [39]:
barrio = pd.get_dummies(cleaning.neighbourhood_cleansed)

In [40]:
cleaning = pd.concat([cleaning, barrio], axis=1)

In [41]:
cleaning.drop('neighbourhood_cleansed', axis=1, inplace=True)

**'latitude' & 'longitude'**: aunque puede parecer un poco confuso y no sabemos si realmente aportará valor al modelo, dejamos estas columnas tal cual están (teniendo en cuenta que no existen valores nulos) y que estas columnas son tipo float.

In [42]:
cleaning.latitude.isnull().sum()

0

In [43]:
cleaning.longitude.isnull().sum()

0

In [44]:
cleaning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4167 entries, 0 to 4166
Data columns (total 44 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   id                                      4167 non-null   int64  
 1   host_since                              4167 non-null   int64  
 2   host_is_superhost                       4167 non-null   int64  
 3   host_total_listings_count               4167 non-null   int64  
 4   host_verifications                      4167 non-null   int64  
 5   host_identity_verified                  4167 non-null   int64  
 6   latitude                                4167 non-null   float64
 7   longitude                               4167 non-null   float64
 8   room_type                               4167 non-null   object 
 9   accommodates                            4167 non-null   int64  
 10  bathrooms_text                          4159 non-null   obje

**'room_type'**: solo existen cuatro posibles valores para esta columna (y no encontramos NaN). Decidimos generar variables ficticias a través de pd.get_dummies()

In [45]:
cleaning.room_type.unique()

array(['Private room', 'Entire home/apt', 'Hotel room', 'Shared room'],
      dtype=object)

In [46]:
room_type = pd.get_dummies(cleaning.room_type)

In [47]:
cleaning = pd.concat([cleaning, room_type], axis=1)

In [48]:
cleaning.drop('room_type', axis=1, inplace=True)

**'accommodates'**: indica la capacidad de huéspedes. No hay problemas aparentes en los datos (más allá de dos valores 0; lo cual no tiene sentido), por lo que dejamos la columna como está.

In [49]:
cleaning.accommodates.isnull().sum()

0

In [50]:
cleaning.accommodates.unique()

array([ 3,  4,  2,  6,  1,  5, 14,  7, 16, 12,  8,  0, 13, 10])

In [51]:
cleaning.accommodates.value_counts()

2     2196
4     1309
3      302
1      117
6      107
5       79
8       18
7       15
16      13
12       5
14       2
0        2
13       1
10       1
Name: accommodates, dtype: int64

**'bathrooms_text'**: indica la cantidad de baños. Los valores vienen en una string que contiene un dígito con el número de baños y especifica el tipo de baño del que se trata o las condiciones de su uso. No entraremos a valorar el tipo de baño y únicamente extraeremos el dígito contenido en la string.

También encontramos 8 valores NaN. Lo primero que haremos será rellenarlos con 0 (entendemos que si no ponen nada es porque no tienen). Luego estableceremos el resto de valores con el dígito contenido en el texto (en los que no contienen dígitos, el texto hace referencia a un half-bath, por lo que definiremos el valor como 0.5).

In [52]:
cleaning.bathrooms_text.unique()

# Nos damos cuenta de los que no tienen dígitos se correponden con 0.5 bathrooms.

array(['1.5 shared baths', '1 bath', '1 private bath', '1.5 baths',
       '2.5 baths', '1 shared bath', '2 baths', '3.5 shared baths',
       '3 baths', 'Half-bath', 'Shared half-bath', '0 baths', nan,
       '4 shared baths', '3.5 baths', '5 baths', '3 shared baths',
       '0 shared baths', '2 shared baths', 'Private half-bath',
       '4.5 baths', '5.5 baths', '4 baths'], dtype=object)

In [53]:
cleaning.bathrooms_text.isnull().sum()

8

In [54]:
cleaning.bathrooms_text.fillna('0.0', inplace=True)

In [55]:
cleaning.bathrooms_text.isnull().sum()

0

In [56]:
def baños_digit (x):
    try:
        digit = re.findall('\d.?\d?', x)
        return float(digit[0])
    except:
        return 0.5  # Los que no contienen dígitos los definimos como 0.5

In [57]:
cleaning.bathrooms_text = cleaning.bathrooms_text.apply(baños_digit)

In [58]:
cleaning.bathrooms_text.value_counts()

1.0    2517
1.5    1207
2.0     231
2.5      88
3.0      45
0.0      30
0.5      22
3.5      19
4.0       3
5.0       2
4.5       2
5.5       1
Name: bathrooms_text, dtype: int64

In [54]:
cleaning.bathrooms_text.value_counts()

1 bath               1667
1.5 baths            1077
1 private bath        601
1 shared bath         249
2 baths               224
1.5 shared baths      130
2.5 baths              88
3 baths                41
0 baths                18
3.5 baths              14
Shared half-bath       13
2 shared baths          7
Half-bath               5
3.5 shared baths        5
3 shared baths          4
0 shared baths          4
Private half-bath       4
5 baths                 2
4.5 baths               2
4 baths                 2
4 shared baths          1
5.5 baths               1
Name: bathrooms_text, dtype: int64

In [93]:
cleaning.bathrooms_text.isnull().sum()

0

In [82]:
baños_digit('hola buenas que tal como')

IndexError: list index out of range

In [69]:
cleaning.head()

,id,host_since,host_is_superhost,host_total_listings_count,host_verifications,host_identity_verified,latitude,longitude,accommodates,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,number_of_reviews,first_review,last_review,review_scores_rating,instant_bookable,reviews_per_month,Bijlmer-Centrum,Bijlmer-Oost,Bos en Lommer,Buitenveldert - Zuidas,Centrum-Oost,Centrum-West,De Aker - Nieuw Sloten,De Baarsjes - Oud-West,De Pijp - Rivierenbuurt,Gaasperdam - Driemond,Geuzenveld - Slotermeer,IJburg - Zeeburgereiland,Noord-Oost,Noord-West,Oostelijk Havengebied - Indische Buurt,Osdorp,Oud-Noord,Oud-Oost,Slotervaart,Watergraafsmeer,Westerpark,Zuid,Entire home/apt,Hotel room,Private room,Shared room
0,0,2012,0,13,8,1,52.39508,4.99186,3,1.5 shared baths,1.0,NaN,"[""First aid kit"", ""Free parking on premises"", ...",87.0,2,81,2014-08-03,2019-06-15,4.62,f,0.91,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,2015,0,0,3,0,52.36371,4.90745,4,1 bath,2.0,2.0,"[""Hot water kettle"", ""Pocket wifi"", ""Stove"", ""...",250.0,2,9,2020-01-26,2021-09-05,5.00,f,0.40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,2,2018,1,20,2,1,52.35482,4.85329,2,1 private bath,1.0,1.0,"[""Air conditioning"", ""First aid kit"", ""Hangers...",151.0,1,47,2018-11-09,2020-03-07,4.79,t,1.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
3,3,2016,0,1,3,0,52.36446,4.85972,2,1.5 baths,1.0,1.0,"[""Hair dryer"", ""Stove"", ""Dishes and silverware...",139.0,2,10,2019-04-22,2019-11-10,5.00,f,0.31,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,4,2013,1,1,9,1,52.34176,4.84782,2,1.5 baths,1.0,2.0,"[""Hair dryer"", ""Dishes and silverware"", ""Ether...",95.0,2,166,2014-04-08,2019-08-26,4.67,f,1.78,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [40]:
cleaning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4167 entries, 0 to 4166
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         4167 non-null   int64  
 1   host_since                 4167 non-null   int64  
 2   host_is_superhost          4167 non-null   int64  
 3   host_total_listings_count  4167 non-null   int64  
 4   host_verifications         4167 non-null   int64  
 5   host_identity_verified     4167 non-null   int64  
 6   neighbourhood_cleansed     4167 non-null   object 
 7   latitude                   4167 non-null   float64
 8   longitude                  4167 non-null   float64
 9   room_type                  4167 non-null   object 
 10  accommodates               4167 non-null   int64  
 11  bathrooms_text             4159 non-null   object 
 12  bedrooms                   3914 non-null   float64
 13  beds                       3993 non-null   float